# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up graphlab create

In [1]:
import turicreate

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = turicreate.SFrame('kc_house_data.csv')

Finished parsing file /home/ppratikbp/Coursera Machine Learning/C2_5_Lasso Regression/kc_house_data.csv

Parsing completed. Parsed 100 lines in 0.12085 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,str,float,int,float,int,int,float,int,int,int,int,int,int,int,int,int,float,float,int,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /home/ppratikbp/Coursera Machine Learning/C2_5_Lasso Regression/kc_house_data.csv

Parsing completed. Parsed 21613 lines in 0.11662 secs.

# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [6]:
model_all = turicreate.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 1.309682     | 6869667.894269     | 385998.458664                   |

| 10        | 11       | 0.000002  | 1.415590     | 6842123.230167     | 364204.576307                   |

+-----------+----------+-----------+--------------+--------------------+---------------------------------+

Completed (Iteration limit reached).

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [47]:
# non_zero_weight = model_all.get("coefficients")["value"]
non_zero_weight = model_all.coefficients
non_zero_weight = non_zero_weight[non_zero_weight['value'] > 0]
print(non_zero_weight)
non_zero_weight.print_rows(num_rows=20)

+------------------+-------+--------------------+--------+
|       name       | index |       value        | stderr |
+------------------+-------+--------------------+--------+
|   (intercept)    |  None | 274873.0556516682  |  None  |
|    bathrooms     |  None | 8468.531052082479  |  None  |
|   sqft_living    |  None | 24.42072110374888  |  None  |
| sqft_living_sqrt |  None | 350.06055100462163 |  None  |
|      grade       |  None | 842.0680288185752  |  None  |
|    sqft_above    |  None | 20.024722643562892 |  None  |
+------------------+-------+--------------------+--------+
[? rows x 4 columns]
Note: Only the head of the SFrame is printed. This SFrame is lazily evaluated.
You can use sf.materialize() to force materialization.
+------------------+-------+--------------------+--------+
|       name       | index |       value        | stderr |
+------------------+-------+--------------------+--------+
|   (intercept)    |  None | 274873.0556516682  |  None  |
|    bathrooms     

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [48]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [49]:
import numpy as np
import pprint 

validation_rss = {}
for l1_penalty in np.logspace(1, 7, num=13):
    model = turicreate.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose = False,
                                              l2_penalty=0., l1_penalty=l1_penalty)
    predictions = model.predict(validation)
    residuals = validation['price'] - predictions
    rss = sum(residuals**2)
    validation_rss[l1_penalty] = rss

# pprint.pprint(result_dict)
print(min(validation_rss.items(), key=lambda x: x[1])) 

(10.0, 625766294657733.5)


*** QUIZ QUESTIONS ***
1. What was the best value for the `l1_penalty`?
2. What is the RSS on TEST data of the model with the best `l1_penalty`?

In [50]:
model_test = turicreate.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose = False,
                                              l2_penalty=0., l1_penalty=10.0)
predictions_test = model.predict(testing)
residuals_test = testing['price'] - predictions_test
rss_test = sum(residuals_test**2)
print(rss_test)

156972768556572.28


***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [51]:
non_zero_weight_test = model_test.coefficients
non_zero_weight_test = non_zero_weight_test[non_zero_weight_test['value'] > 0]
print(non_zero_weight_test)
non_zero_weight_test.print_rows(num_rows=20)

+------------------+-------+----------------------+--------+
|       name       | index |        value         | stderr |
+------------------+-------+----------------------+--------+
|   (intercept)    |  None |  18993.42639378385   |  None  |
|     bedrooms     |  None |  7936.967506315497   |  None  |
| bedrooms_square  |  None |  936.9933634355792   |  None  |
|    bathrooms     |  None |  25409.589025389036  |  None  |
|   sqft_living    |  None |  39.115136954825466  |  None  |
| sqft_living_sqrt |  None |  1124.650213097026   |  None  |
|     sqft_lot     |  None | 0.003483624587288124 |  None  |
|  sqft_lot_sqrt   |  None |  148.2583942897513   |  None  |
|      floors      |  None |  21204.335313065483  |  None  |
|  floors_square   |  None |  12915.524437616072  |  None  |
+------------------+-------+----------------------+--------+
[? rows x 4 columns]
Note: Only the head of the SFrame is printed. This SFrame is lazily evaluated.
You can use sf.materialize() to force material

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [52]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [54]:
l1_penalty_values = np.logspace(8, 10, num=20)
print(l1_penalty_values)

[1.00000000e+08 1.27427499e+08 1.62377674e+08 2.06913808e+08
 2.63665090e+08 3.35981829e+08 4.28133240e+08 5.45559478e+08
 6.95192796e+08 8.85866790e+08 1.12883789e+09 1.43844989e+09
 1.83298071e+09 2.33572147e+09 2.97635144e+09 3.79269019e+09
 4.83293024e+09 6.15848211e+09 7.84759970e+09 1.00000000e+10]


Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [63]:
coef_dict = {}
for l1_penalty in l1_penalty_values:
    model = turicreate.linear_regression.create(training, target ='price', features=all_features,
                                              validation_set=None, verbose=None,
                                              l2_penalty=0., l1_penalty=l1_penalty)
    coef_dict[l1_penalty] = model.coefficients
    coef_dict[l1_penalty] = coef_dict[l1_penalty][coef_dict[l1_penalty]['value']]

In [64]:
pprint.pprint(coef_dict)

{100000000.0: Columns:
	name	str
	index	str
	value	float
	stderr	float

Rows: Unknown

Data:
+------------------+-------+-----------------------+--------+
|       name       | index |         value         | stderr |
+------------------+-------+-----------------------+--------+
|   (intercept)    |  None |   25090.91655983156   |  None  |
|     bedrooms     |  None |   7789.176891636114   |  None  |
| bedrooms_square  |  None |   847.5596826532682   |  None  |
|    bathrooms     |  None |   25234.209290943505  |  None  |
|   sqft_living    |  None |   39.039446544242494  |  None  |
| sqft_living_sqrt |  None |   1117.3118961277494  |  None  |
|     sqft_lot     |  None | -0.025686122351637274 |  None  |
|  sqft_lot_sqrt   |  None |    143.988995456053   |  None  |
|      floors      |  None |   20695.35909265733   |  None  |
|  floors_square   |  None |   12466.690754862868  |  None  |
+------------------+-------+-----------------------+--------+
[? rows x 4 columns]
Note: Only the hea

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzero` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzero` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [65]:
l1_penalty_min = 2976351441.6313128
l1_penalty_max = 3792690190.7322536

***QUIZ QUESTIONS***

What values did you find for `l1_penalty_min` and`l1_penalty_max`? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [66]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzero`.

In [74]:
validation_rss = {}
for l1_penalty in l1_penalty_values:
    model = turicreate.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose = False,
                                              l2_penalty=0., l1_penalty=l1_penalty)
    predictions = model.predict(validation)
    residuals = validation['price'] - predictions
    rss = sum(residuals**2)
    model_coef = model.coefficients
    model_coef = model_coef['value']
    validation_rss[l1_penalty] = rss, model_coef
    

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [95]:
model = turicreate.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose = False,
                                              l2_penalty=0., l1_penalty=3448968612.16)

In [99]:
non_zero_weight_test = model.coefficients
non_zero_weight_test = non_zero_weight_test[non_zero_weight_test['value'] > 0]
non_zero_weight_test.print_rows(num_rows=8)

+------------------+-------+--------------------+--------+
|       name       | index |       value        | stderr |
+------------------+-------+--------------------+--------+
|   (intercept)    |  None | 222253.19177946472 |  None  |
|     bedrooms     |  None | 661.7225625501316  |  None  |
|    bathrooms     |  None | 15873.957390454756 |  None  |
|   sqft_living    |  None | 32.41022208406935  |  None  |
| sqft_living_sqrt |  None | 690.1147756379333  |  None  |
|      grade       |  None | 2899.420271124415  |  None  |
|    sqft_above    |  None | 30.011576020981668 |  None  |
+------------------+-------+--------------------+--------+
[7 rows x 4 columns]

